# Sveobuhvatni zadatak iz AC krugova

Ovo je Jupyter (Python) bilježnica (notebook) za generiranje personaliziranih parametara i rješenja (npr. po učeniku, za puno njih od jednom) za tzv. "sveobuhvatni zadatak" iz AC krugova, kojeg je osmislio i napisao (kao i ovu "bilježnicu") **Eugen Rožić**, prof. (17.7.2025.).

Dokument sa zadatkom (i druge popratne materijale, kao i između ostalog ovaj notebook) trebali bi moći pronaći na mojoj web stranici (u trenutku u kojem ovo pišem adresa web stranice je __[erozic.github.io](https://erozic.github.io)__)

##### Upute za korištenje
Svaku od ćelija potrebno je izvršiti (_shift+Enter_) barem jednom, neke od njih i prilagoditi. Svaka ćelija sadrži komentare (iza # znakova) koji sadrže daljenje upute i objašnjenja. Ako se njih pažljivo pročita ne bi trebalo biti problema u korištenju, za svakoga tko zna dovoljno ili je dovoljno snalažljiv da je notebook uspio pokrenuti.

Nadam se da će biti od koristi... Sretno :)

In [1]:
import numpy as np
from numpy import conj
from math import sqrt, radians, degrees, pi
from cmath import rect, polar
import random

# Tu se nalaze definicije metoda koje se koriste u ovom notebooku. Nije potrebno ni preporučljivo
# ništa mijenjati ni dirati u ovoj ćeliji (samo ju jednom na početku izvršiti), ali mudro bi bilo
# pročitati i proučiti dokumentaciju (opise tj. objašnjenja metoda na njihovom početku)

def z_deg(z_rad, A_precision = 2, phi_precision = 1):
    """Vraća string reprezentaciju numpy kompleksnog broja u polarnom obliku, u stupnjevima."""
    format_string = "{:."+str(A_precision)+"f}|{:."+str(phi_precision)+"f}°"
    return format_string.format(z_rad[0], degrees(z_rad[1]))

def params_from_string(line):
    """Očekuje redak teksta u formatu:
    string(name), int(f in Hz), float(A in V), float(phi in rad), int(R1 in Ohm), float(R2 in  kOhm), int(L in mH), int(C in nF)
    Vraća par (ime_učenika, dictionary parametara), pri čemu su ključevi dictonaryja nazivi varijabli kako su napisani iznad."""
    parts = line.split()
    params = {'f':int(parts[1]), 'A':float(parts[2]), 'phi':float(parts[3]), 'R1':int(parts[4]), 'R2':float(parts[5]), 'L':int(parts[6]), 'C':int(parts[7])}
    return parts[0], params

def student_params_from_table(table_string):
    """Vraća dictionary parametara po učeniku iz tekstualnog tabličnog zapisa (npr. direktni c/p in worda ili pdfa).
    Kao input očekuje podatke za svakog učenika u jednom tekstualnom retku (line) i da su podatci u retku odvojeni
    nekim whitespace znakom ili više njih (razmak, tab). Svaki redak mora biti formata kako ga očekuje metoda
    params_from_string.
    Vraća dictonary gdje su ključevi imena učenika, a vrijednosti dictionaryji parametara za svakog učenika (kako ih
    vraća metoda params_from_string)."""
    student_params = {}
    for line in table_string.splitlines():
        name, params = params_from_string(line)
        student_params[name] = params
    return student_params

def random_params():
    """Vraća nasumični skup parametara, kao dictionary."""
    params = {}
    params['f'] = random.choice(f_range) #Hz
    params['A'] = random.choice(A_range) #V
    params['phi'] = random.choice(phi_range) #rad
    params['R1'] = random.choice(R1_range) #Ohm
    params['R2'] = random.choice(R2_range) #kOhm
    params['L'] = random.choice(L_range) #mH
    params['C'] = random.choice(C_range) #nF
    return params

def solve(params):
    """Riješava zadatak za skup parametara (dictionary), vraća uređenu 27-orku (tuple)."""
    R1 = params['R1'] #in Ohm
    R2 = params['R2']*10**3 #in Ohm
    L = params['L']*10**-3 #in H
    C = params['C']*10**-9 #in F
    w = 2*pi*params['f']
    U_ef = params['A']/sqrt(2)
    U = rect(U_ef, params['phi'])
    Z_L = w*L*1j
    Z_C = 1/(w*C*1j)
    Z1 = R1 + Z_L
    Z2 = 1/((1/R2) + (1/Z_C))
    Z_uk = Z1 + Z2
    I_uk = U/Z_uk
    U_L = Z_L*I_uk
    U_R1 = R1*I_uk
    U_Z1 = Z1*I_uk
    U_Z2 = U - U_Z1
    I_C = U_Z2/Z_C
    I_R2 = U_Z2/R2
    S_uk = U*conj(I_uk)
    P_uk = S_uk.real
    Q_uk = S_uk.imag
    P_R1 = abs(I_uk)**2*R1
    P_R2 = abs(I_R2)**2*R2
    Q_L = abs(I_uk)**2*Z_L.imag
    Q_C = abs(I_C)**2*Z_C.imag

    Ck = Q_uk/(w*abs(U)**2)
    if Ck < 0:
        return None
    FS = P_uk/abs(S_uk)

    Ck_ = 0.8*Ck
    Q_Ck = -w*Ck_*abs(U)**2
    Q_uk_ = Q_uk + Q_Ck
    S_uk_ = complex(P_uk, Q_uk_)
    FS_ = P_uk/abs(S_uk_)
    
    if (1/(L*C) - 1/(R2*C)**2) < 0:
        return None
    w_rez = sqrt(1/(L*C) - 1/(R2*C)**2)
    f_rez = w_rez/(2*pi)
    
    return U, w, Z_L, Z_C, Z1, Z2, Z_uk, I_uk, U_L, U_R1, U_Z1, U_Z2, I_C, I_R2, \
    S_uk, P_uk, Q_uk, P_R1, P_R2, Q_L, Q_C, Ck, FS, Ck_, S_uk_, FS_, w_rez, f_rez

def print_solution(U, w, Z_L, Z_C, Z1, Z2, Z_uk, I_uk, U_L, U_R1, U_Z1, U_Z2, I_C, I_R2, \
    S_uk, P_uk, Q_uk, P_R1, P_R2, Q_L, Q_C, Ck, FS, Ck_, S_uk_, FS_, w_rez, f_rez):
    """Ispisuje uredno i lijepo ono što metoda "solve" vraća."""
    print("=====================================")
    print("U = {} V = {:.1f} + {:.1f}j".format(z_deg(polar(U),1,1), U.real, U.imag))
    print("w = {:.0f} rad/s".format(w))
    print("Z_L = {:.0f}j Ohm".format(Z_L.imag))
    print("Z_C = {:.0f}j Ohm".format(Z_C.imag))
    print()
    print("Z1 = {} Ohm = {:.1f} + {:.1f}j".format(z_deg(polar(Z1),1,1), Z1.real, Z1.imag))
    print("Z2 = {} Ohm = {:.1f} + {:.1f}j".format(z_deg(polar(Z2),1,1), Z2.real, Z2.imag))
    print("Z_uk = {} Ohm = {:.1f} + {:.1f}j".format(z_deg(polar(Z_uk),1,1), Z_uk.real, Z_uk.imag))
    print("I_uk = {} mA = {:.1f} + {:.1f}j".format(z_deg(polar(1000*I_uk),1,1), I_uk.real*1000, I_uk.imag*1000))
    print()
    print("U_L = {} V = {:.2f} + {:.2f}j".format(z_deg(polar(U_L),2,1), U_L.real, U_L.imag))
    print("U_R1 = {} V = {:.2f} + {:.2f}j".format(z_deg(polar(U_R1),2,1), U_R1.real, U_R1.imag))
    print("U_Z1 = {} V = {:.2f} + {:.2f}j".format(z_deg(polar(U_Z1),2,1), U_Z1.real, U_Z1.imag))
    print("U_Z2 = {} V = {:.2f} + {:.2f}j".format(z_deg(polar(U_Z2),2,1), U_Z2.real, U_Z2.imag))
    print("I_C = {} mA = {:.2f} + {:.2f}j".format(z_deg(polar(1000*I_C),2,1), I_C.real*1000, I_C.imag*1000))
    print("I_R2 = {} mA = {:.2f} + {:.2f}j".format(z_deg(polar(1000*I_R2),2,1), I_R2.real*1000, I_R2.imag*1000))
    print()
    print("S_uk = {} mVA = {:.1f} + {:.1f}j".format(z_deg(polar(1000*S_uk),1,1), S_uk.real*1000, S_uk.imag*1000))
    print("P_uk = {:.1f} mW".format(1000*P_uk))
    print("Q_uk = {:.1f} mVAr".format(1000*Q_uk))
    print("P_R1 = {:.1f} mW".format(1000*P_R1))
    print("P_R2 = {:.1f} mW".format(1000*P_R2))
    print("Q_L = {:.1f} mVAr".format(1000*Q_L))
    print("Q_C = {:.1f} mVAr".format(1000*Q_C))
    print("=====================================")
    print("Ck = {:.2f} nF".format(Ck*10**9))
    print("faktor snage = {:.1f} %".format(100*FS))
    print("=====================================")
    print("Ck' = {:.2f} nF".format(Ck_*10**9))
    print("S_uk' = {} mVA = {:.1f} + {:.1f}j".format(z_deg(polar(1000*S_uk_),1,1), S_uk_.real*1000, S_uk_.imag*1000))
    print("faktor snage = {:.1f} %".format(100*FS_))
    print("=====================================")
    print("w_rez = {:.1f} rad/s".format(w_rez))
    print("f_rez = {:.1f} Hz".format(f_rez))


In [2]:
# Tu se nalaze parametri koje možete (a i ne morate) mijenjati/prilagođavati vašim potrebama...

# Ovo je skup parametara koji je u originalnom tekstu zadatka, stavljen u format u kojem
# ga očekuju i koriste metode u ovom notebooku, a to je dictionary sa sljedećim ključevima:
# f[Hz], A[V], phi[rad], R1[Ohm], R2[kOhm], L[mH], C[nF] (u zagradama su očekivane mjerne jedinice)
default_params = {'f':500,
                  'A':35.35,
                  'phi':1.234,
                  'R1':500,
                  'R2':1.5,
                  'L':420,
                  'C':210}

# Tu su definirani rasponi (i koraci) iz kojih se generiraju nasumični parametri za svakog učenika (za metodu random_params)
f_range = np.arange(350, 650, 10) # f_min, f_max, f_step u Hz
A_range = np.arange(20, 50, 1) # A_min, A_max, A_step u V
phi_range = np.round(np.arange(3*np.pi/18, 33*np.pi/18, np.pi/18), 2) # phi_min, phi_max, phi_step u rad
R1_range = np.arange(350, 650, 10) # R1_min, R1_max, R1_step u Ohm
R2_range = np.round(np.arange(1.50, 3.00, 0.05), 2) # R2_min, R2_max, R2_step u kOhm
L_range = np.arange(300, 600, 10) # L_min, L_max, L_step u mH
C_range = np.arange(150, 300, 5) # C_min, C_max, C_step u nF

In [3]:
# Tu se nalazi popis imena učenika za koje želite generirati skupove parametara (personalizirane zadatke) ...
# OPREZ: ime/oznaka za učenika treba biti jedna riječ, koja ne sadrži razmake
students = [
    "Učenik_1",
    "Učenik_2" # itd.
]
# Nakon što promijenite popis nemojte zaboraviti ponovno izvršiti (shift+Enter) ovaj okvir/ćeliju

In [4]:
# Izvršavanjem ove ćelije generiraju se nasumični skupovi parametara za svakog učenika s popisa

# Tekstualni ispis koji će ćelija generirati ispod možete copy/pasteati npr. u Word tablicu ili
# u neki tekstualni dokument koji onda možete podijeliti s učenicima...
# SAVJET: Ispis ćelije možete doslovno kopirat/zalijepit u Word dokument u obliku običnog teksta,
# i potom taj tekst označiti i odabrati opciju "Umetanje->Tablica->Pretvori tekst u tablicu..."

student_params = {}
for student in students:
    params = random_params()
    solution = solve(params)
    while solution == None:
        print("no solution for params: f={:d}  A={:.1f}  phi={:.2f}  R1={:d}  R2={:.2f}  L={:d}  C={:d}".format(
            params['f'], params['A'], params['phi'], params['R1'], params['R2'], params['L'], params['C']))
        params = random_params()
        solution = solve(params)
    student_params[student] = params

print()
print("{:10s}\t{:5s}\t{:4s}\t{:8s}\t{:7s}\t{:8s}\t{:5s}\t{:5s}".format(
    "Prezime", "f[Hz]", "A[V]", "phi[rad]", "R1[Ohm]", "R2[kOhm]", "L[mH]", "C[nF]"))
for student, params in student_params.items():
    print("{:10s}\t{:5d}\t{:4.1f}\t{:8.2f}\t{:7d}\t{:8.2f}\t{:5d}\t{:5d}".format(
        student, params['f'], params['A'], params['phi'], params['R1'], params['R2'], params['L'], params['C']))

no solution for params: f=390  A=44.0  phi=3.84  R1=540  R2=2.90  L=350  C=175

Prezime   	f[Hz]	A[V]	phi[rad]	R1[Ohm]	R2[kOhm]	L[mH]	C[nF]
Učenik_1  	  350	41.0	    5.24	    500	    1.80	  490	  240
Učenik_2  	  610	22.0	    4.71	    590	    2.00	  300	  270


In [5]:
# Tu se nalazi tablica parametara po učeniku u tekstualnom obliku, npr. copy/paste od ispisa
# ćelije iznad ili iz tablice (istog formata) iz nekog Word ili PDF dokumenta itd.

# To služi tome da bi se ispod mogla generirati rješenja (npr. vama za provjeru njihovih rješenja)
# iz već ranije generiranih skupova parametara za svakog učenika.

# Nakon što ispod kopirate (ili ručno upišete) učenike i njihove parametre, izvršite ovu ćeliju da bi
# mogli generirati rješenja...

students_raw = """Učenik_1  	  620	49.0	    5.41	    630	    2.50	  300	  270
Učenik_2  	  530	40.0	    1.22	    410	    2.70	  500	  200""" # itd.
student_params = student_params_from_table(students_raw)

In [6]:
# Pokretanjem ove ćelije ispisati će se rješenje zadatka za jednog učenika, tj. za jedan skup parametara

#print_solution(*solve(default_params)) # za testiranje
print_solution(*solve(student_params['Učenik_1']))

U = 34.6|-50.0° V = 22.3 + -26.6j
w = 3896 rad/s
Z_L = 1169j Ohm
Z_C = -951j Ohm

Z1 = 1327.7|61.7° Ohm = 630.0 + 1168.7j
Z2 = 888.7|-69.2° Ohm = 315.9 + -830.6j
Z_uk = 1004.5|19.7° Ohm = 945.9 + 338.1j
I_uk = 34.5|-69.7° mA = 12.0 + -32.4j

U_L = 40.31|20.3° V = 37.81 + 13.99j
U_R1 = 21.73|-69.7° V = 7.54 + -20.38j
U_Z1 = 45.80|-8.0° V = 45.35 + -6.39j
U_Z2 = 30.65|-138.9° V = -23.09 + -20.16j
I_C = 32.24|-48.9° mA = 21.21 + -24.29j
I_R2 = 12.26|-138.9° mA = -9.24 + -8.06j

S_uk = 1195.1|19.7° mVA = 1125.4 + 402.2j
P_uk = 1125.4 mW
Q_uk = 402.2 mVAr
P_R1 = 749.6 mW
P_R2 = 375.8 mW
Q_L = 1390.5 mVAr
Q_C = -988.3 mVAr
Ck = 86.01 nF
faktor snage = 94.2 %
Ck' = 68.81 nF
S_uk' = 1128.3|4.1° mVA = 1125.4 + 80.4j
faktor snage = 99.7 %
w_rez = 3186.0 rad/s
f_rez = 507.1 Hz
